## 多输入通道和多输出通道
### 多输入通道

In [1]:
import torch
from torch import nn
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

def corr2d_multi_in(X, K):
    # 沿X和K的第0维分别计算 再相加  也就是相同通道的一起计算
    res = d2l.corr2d(X[0, :, :], K[0, :, :])
    for i in range(1, X.shape[0]):
        # 遍历通道
        res += d2l.corr2d(X[i, :, :], K[i, :, :])
    return res

In [2]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 多输出通道

In [3]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同输入X做互相关计算。所有结果使用stack函数合并在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K])

In [4]:
K = torch.stack([K, K + 1, K + 2])
K.shape, K # torch.Size([3, 2, 2, 2])

(torch.Size([3, 2, 2, 2]),
 tensor([[[[0, 1],
           [2, 3]],
 
          [[1, 2],
           [3, 4]]],
 
 
         [[[1, 2],
           [3, 4]],
 
          [[2, 3],
           [4, 5]]],
 
 
         [[[2, 3],
           [4, 5]],
 
          [[3, 4],
           [5, 6]]]]))

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 1x1卷积层

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.view(c_i, h * w)
    K = K.view(c_o, c_i)
    Y = torch.mm(K, X)  # 全连接层的矩阵乘法
    return Y.view(c_o, h, w)

In [8]:
X = torch.rand(3, 3, 3)
K = torch.rand(2, 3, 1, 1)

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

(Y1 - Y2).norm().item() < 1e-6


True